In [187]:
import pandas as pd
import os
import ast
import re

!pip install python-Levenshtein
import Levenshtein


The folder you are executing pip from can no longer be found.


In [188]:
def extract_text(text):
    try:
        dict_obj = ast.literal_eval(text)
        return dict_obj['text']
    except (ValueError, KeyError):
        return pd.NA


In [189]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]','',text) #removing special character and digits
    text = text.lower()
    return text

In [190]:
def merge_and_preprocess_df(path1,path2):
    tran_df = pd.read_csv(path1)
    tran_df.drop(columns=['Subset_Name'], inplace=True)
    tran_df = tran_df.rename(columns={'File': 'file_name'})
    
    metadata_df = pd.read_csv(path2)
    metadata_df = metadata_df[['file_name','phrase','merged_prompt']]
    metadata_df.head()
    

    final_df = pd.merge(tran_df, metadata_df, on='file_name', how='inner')
    final_df['Transcription'] = final_df['Transcription'].apply(extract_text)
    final_df['Transcription'] = final_df['Transcription'].apply(preprocess_text)
    return final_df

In [191]:
metadata_path = '../../../data/metadata'
print(os.listdir(train_metadata_path))

meta_train = '../../../data/metadata/metadata_train.csv'
meta_test = '../../../data/metadata/metadata_test.csv'
meta_validate = '../../../data/metadata/metadata_validate.csv'

['metadata_train.csv', 'metadata_test.csv', 'metadata_validate.csv']


In [192]:
train_path = '/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/transcription_train_whisper-ai.csv'
test_path = '/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/transcriptions_test_whisper-ai.csv'
validate_path = '/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/transcriptions_validate_whisper-ai.csv'

In [193]:
transcription_train = merge_and_preprocess_df(train_path,meta_train)
transcription_train.head()

,file_name,Transcription,phrase,merged_prompt
0,1249120_15004831_103229689.wav,my son had his lip pierced and it is swollen ...,my son had his lip pierced and it is swollen a...,Wound and injury
1,1249120_15004831_93767590.wav,my back hurts so much i cant bend down to tie...,my back hurts so much i cant bend down to tie ...,"Neck, back or spinal issues"
2,1249120_15830408_55921206.wav,i was travelling by ship and i feel dizzy,i was travelling by ship and i feel dizzy,Dizziness and vertigo
3,1249120_15830408_37803938.wav,i do not know why i feel cold,i do not know why i feel cold,Feeling cold/hot
4,1249120_2171654_19438090.wav,my hair is falling out after i take a shower,my hair is falling out after i take a shower,Hair and skin issues


In [194]:
transcription_validate = merge_and_preprocess_df(validate_path,meta_validate)
transcription_validate.head()

,file_name,Transcription,phrase,merged_prompt
0,1249120_44246595_89466082.wav,i often get a stomach ache after i eat i have...,i often get a stomach ache after i eat i have...,Digestive issues
1,1249120_44246595_101823153.wav,i feel pain in my right shoulder after the te...,i feel pain in my right shoulder after the ten...,Wound and injury
2,1249120_44246595_94621639.wav,i feel pain inside i do not know what it is,i feel pain inside i do not know what it is,Internal pain
3,1249120_44246595_89680770.wav,i dont have the energy to do the things that ...,i dont have the energy to do the things that i...,General weakness
4,1249120_44246595_107248522.wav,i notice an important decrease in my vision w...,i noticed an important decrease in my vision w...,Sensory issues


In [195]:
transcription_test = merge_and_preprocess_df(test_path,meta_test)
transcription_test.head()

,file_name,Transcription,phrase,merged_prompt
0,1249120_44142156_102691000.wav,my knee feels weak and it gave away the other...,my knee feels weak and it gave way the other d...,Leg and foot pain
1,1249120_44142156_109272216.wav,i think i sprained my ankle it really hurts,i think i sprained my ankle it really hurts,Wound and injury
2,1249120_44142156_103207247.wav,is my cut infected or just healing,is my cut infected or just healing,Wound and injury
3,1249120_44142156_100535941.wav,i have some terrible problems when i am breat...,i have some terrible problems when im breathin...,Respiratory issue
4,1249120_44142156_65967262.wav,if i hit my head at the basketball game could...,i hit my head at the basketball game could i ...,Wound and injury


In [196]:
def wer(reference, hypothesis):
    """
    Calculate Word Error Rate (WER) between reference and hypothesis.
    """
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    # Calculate Levenshtein distance
    distance = Levenshtein.distance(reference_words, hypothesis_words)

    # Calculate WER
    wer = distance / len(reference_words)
    return round(wer,4)


def cer(reference, hypothesis):
    """
    Calculate Character Error Rate (CER) between reference and hypothesis.
    """
    # Calculate Levenshtein distance
    distance = Levenshtein.distance(reference, hypothesis)

    # Calculate CER
    cer = distance / len(reference)
    return round(cer,4)

In [197]:
wer_scores = []
cer_scores = []

for actual, pred in zip(transcription_train['phrase'], transcription_train['Transcription']):
    wer_scores.append(wer(actual, pred))
    cer_scores.append(cer(actual, pred))

In [198]:
transcription_train['word_error_rate'] = wer_scores
transcription_train['character_error_rate'] = cer_scores

In [199]:
transcription_train

,file_name,Transcription,phrase,merged_prompt,word_error_rate,character_error_rate
0,1249120_15004831_103229689.wav,my son had his lip pierced and it is swollen ...,my son had his lip pierced and it is swollen a...,Wound and injury,0.0455,0.0196
1,1249120_15004831_93767590.wav,my back hurts so much i cant bend down to tie...,my back hurts so much i cant bend down to tie ...,"Neck, back or spinal issues",0.0000,0.0172
2,1249120_15830408_55921206.wav,i was travelling by ship and i feel dizzy,i was travelling by ship and i feel dizzy,Dizziness and vertigo,0.0000,0.0244
3,1249120_15830408_37803938.wav,i do not know why i feel cold,i do not know why i feel cold,Feeling cold/hot,0.0000,0.0345
4,1249120_2171654_19438090.wav,my hair is falling out after i take a shower,my hair is falling out after i take a shower,Hair and skin issues,0.0000,0.0227
...,...,...,...,...,...,...
495,1249120_17632529_59148406.wav,im feeling cold though the temperature is high,i m feeling cold though the temperature is high,Feeling cold/hot,0.2222,0.0426
496,1249120_15004831_58758228.wav,i have an earache that wont go away i dont ha...,i have an ear ache that wont go away i dont h...,Sensory issues,0.0870,0.0374
497,1249120_14353703_42564912.wav,i usually have a bad cough when i get flu,i usually have a bad cough when i get flu,Respiratory issue,0.0000,0.0244
498,1249120_15004831_109812806.wav,i cant see the sign i have a blurry vision,i cant see the sign i have a blurry vision,Sensory issues,0.0000,0.0238


In [200]:
print('Mean WER:', round(transcription_train['word_error_rate'].mean(), 4))

Mean WER: 0.0413


In [201]:
print('Mean CER:', round(transcription_train['character_error_rate'].mean(), 4))

Mean CER: 0.0406


In [202]:
# transcription_train.to_csv('/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/train_recordings_transcription_whisper-ai.csv', index=False)

## TEST

In [203]:
wer_scores = []
cer_scores = []

for actual, pred in zip(transcription_test['phrase'], transcription_test['Transcription']):
    wer_scores.append(wer(actual, pred))
    cer_scores.append(cer(actual, pred))

transcription_test['word_error_rate'] = wer_scores
transcription_test['character_error_rate'] = cer_scores


print('Mean WER:', round(transcription_test['word_error_rate'].mean(), 4))

print('Mean CER:', round(transcription_test['character_error_rate'].mean(), 4))

# recordings_df.to_csv('./train_recordings_transcription_AWS.csv', index=False)

Mean WER: 0.0752
Mean CER: 0.0554


In [204]:
# transcription_test.to_csv('/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/test_recordings_transcription_whisper-ai.csv', index=False)

## Validate

In [205]:
wer_scores = []
cer_scores = []

for actual, pred in zip(transcription_validate['phrase'], transcription_validate['Transcription']):
    wer_scores.append(wer(actual, pred))
    cer_scores.append(cer(actual, pred))

transcription_validate['word_error_rate'] = wer_scores
transcription_validate['character_error_rate'] = cer_scores


print('Mean WER:', round(transcription_validate['word_error_rate'].mean(), 4))

print('Mean CER:', round(transcription_validate['character_error_rate'].mean(), 4))


Mean WER: 0.4259
Mean CER: 0.3463


In [206]:
transcription_validate

,file_name,Transcription,phrase,merged_prompt,word_error_rate,character_error_rate
0,1249120_44246595_89466082.wav,i often get a stomach ache after i eat i have...,i often get a stomach ache after i eat i have...,Digestive issues,0.0476,0.0583
1,1249120_44246595_101823153.wav,i feel pain in my right shoulder after the te...,i feel pain in my right shoulder after the ten...,Wound and injury,0.0000,0.0182
2,1249120_44246595_94621639.wav,i feel pain inside i do not know what it is,i feel pain inside i do not know what it is,Internal pain,0.0000,0.0233
3,1249120_44246595_89680770.wav,i dont have the energy to do the things that ...,i dont have the energy to do the things that i...,General weakness,0.0000,0.0185
4,1249120_44246595_107248522.wav,i notice an important decrease in my vision w...,i noticed an important decrease in my vision w...,Sensory issues,0.0588,0.0244
...,...,...,...,...,...,...
380,1249120_44387284_64093494.wav,whats the way over whats the magic,i was injured during football match i was diag...,Wound and injury,1.0000,0.7568
381,1249120_44415714_12774994.wav,i think its hot enough,i read a book for along time and when i finish...,NaN,0.9375,0.8356
382,1249120_44442354_57228239.wav,my skiing is the chain,my skin is itching,Hair and skin issues,0.7500,0.5000
383,1249120_44442354_96518720.wav,i film the state net in my elbows and shoulder,i fell a stiffness in my elbows and shoulders,Muscle and joint pain,0.5556,0.2889


In [207]:
# transcription_validate.to_csv('/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/validate_recordings_transcription_whisper-ai.csv', index=False)

## Comparing AWS and Whisper-AI

In [208]:
aws_file_path = "/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/aws/train_recordings_transcription_AWS.csv"
aws_df = pd.read_csv(aws_file_path)
aws_df.head()

,file_name,actual_transcription,pred_transcription,word_error_rate,character_error_rate
0,1249120_13842059_104469105.wav,I have a painful cramp in my feet,I have a painful cramp in my feet.,0.1250,0.0303
1,1249120_13842059_105045085.wav,The pain feels like it's right below the skin,The pain feels like it's right below the skin.,0.1111,0.0222
2,1249120_13842059_11964685.wav,I feel suicidal.,I feel suicidal.,0.0000,0.0000
3,1249120_13842059_12420758.wav,I feel a sharp pain in my ankle joint when I s...,I feel a sharp pain in my ankle joint when I s...,0.0000,0.0000
4,1249120_13842059_13041979.wav,My shoulder hurts me so much,My shoulder hurts me so much.,0.1667,0.0357


In [209]:
print('Mean WER:', round(aws_df['word_error_rate'].mean(), 4))

print('Mean CER:', round(aws_df['character_error_rate'].mean(), 4))

Mean WER: 0.181
Mean CER: 0.0467


In [210]:
whisper_file_path = "/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/wer_cer_scores/train_recordings_transcription_whisper-ai.csv"
whisper_df = pd.read_csv(whisper_file_path)
whisper_df = whisper_df.rename(columns={'Transcription': 'pred_transcription', 'phrase': 'actual_transcription'})
whisper_df.head()

,file_name,pred_transcription,actual_transcription,merged_prompt,word_error_rate,character_error_rate
0,1249120_15004831_103229689.wav,my son had his lip pierced and it is swollen ...,my son had his lip pierced and it is swollen a...,Wound and injury,0.0455,0.0196
1,1249120_15004831_93767590.wav,my back hurts so much i cant bend down to tie...,my back hurts so much i cant bend down to tie ...,"Neck, back or spinal issues",0.0000,0.0172
2,1249120_15830408_55921206.wav,i was travelling by ship and i feel dizzy,i was travelling by ship and i feel dizzy,Dizziness and vertigo,0.0000,0.0244
3,1249120_15830408_37803938.wav,i do not know why i feel cold,i do not know why i feel cold,Feeling cold/hot,0.0000,0.0345
4,1249120_2171654_19438090.wav,my hair is falling out after i take a shower,my hair is falling out after i take a shower,Hair and skin issues,0.0000,0.0227


In [211]:
print('Mean WER:', round(whisper_df['word_error_rate'].mean(), 4))

print('Mean CER:', round(whisper_df['character_error_rate'].mean(), 4))

Mean WER: 0.0413
Mean CER: 0.0406
